In [2]:
import wave
import struct
import sys
import os
import librosa
import soundfile as sf
import pandas as pd
import numpy as np



In [23]:
def wav_to_floats(wave_file):
    w = wave.open(wave_file)
    astr = w.readframes(w.getnframes())
    framerate = w.getframerate()
    # convert binary chunks to short 
    a = struct.unpack("%ih" % (w.getnframes()* w.getnchannels()), astr)
    a = [float(val) / pow(2, 15) for val in a]
    return a, framerate

# read the wav file specified as first command line arg
signal, framerate = wav_to_floats("../All_wavs/CreepingDeath.wav")
print("read "+str(len(signal))+" frames")
print("sampling rate:" + str(framerate))
print("in the range "+str(min(signal))+" to "+str(max(signal)))
signal, framerate = wav_to_floats("../preprocessed_wavs/CreepingDeath.wav")
print("read "+str(len(signal))+" frames")
print("sampling rate:" + str(framerate))
print("in the range "+str(min(signal))+" to "+str(max(signal)))

read 37987840 frames
sampling rate:48000
in the range -1.0 to 0.999969482421875
read 8725332 frames
sampling rate:22050
in the range -1.0 to 0.999969482421875


In [ ]:
import pandas as pd
import re
#cleans the lyrics
def clean_lyrics(file_path, new_file_path):
    df1 = pd.read_csv(file_path)
    def clean_helper(lyrics):
        if (lyrics == -1 or lyrics == '-1'): return -1
        if lyrics.find('Lyrics') != -1:
            lyrics = lyrics[lyrics.find("Lyrics")+len('Lyrics'):]
        lyrics = (re.sub("\[.*?\]", '', lyrics))
        lyrics = lyrics.replace('Embed', '')
        lyrics = lyrics.replace('You might also like', '')
        regex = re.compile('[^a-zA-Z0-9 \n]')
        lyrics = re.sub(regex, '', lyrics)
        return lyrics.strip()
    df1['Lyrics'] = df1['Lyrics'].apply(clean_helper)
    df1.to_csv(new_file_path, index=False)


In [61]:
#convert full sound wavs into 30 second 22050 htz wavs
list_ = os.listdir("../All_wavs")
for i,val in enumerate(list_):
    y, sr = librosa.load(f"../All_wavs/{val}", sr=22050)
    #we just want 30 seconds of song data from each song, so we will pick the middle 30 seconds of the song
    # we want the middle 661500 samples
    sample_size = 661500
    thirty_sec_song = y[(int)(y.size/2 - sample_size/2): (int)(y.size/2 + sample_size/2)]
    print(thirty_sec_song.shape)
    sf.write(f"../30_second_wavs/{val}", thirty_sec_song, samplerate =22050)


(661500,)
(661500,)
(661500,)
(661500,)
(661500,)
(661500,)
(661500,)
(661500,)
(661500,)
(661500,)


In [12]:
#initialize the test_set with only the wavs
columns = ['Artist', 'Sample_Rate', 'Wav']
df = pd.DataFrame(columns=columns)
list_ = os.listdir("../30_second_wavs")
for i,val in enumerate(list_):
    y, sr = librosa.load(f"../30_second_wavs/{val}", sr=22050)
    df = df._append({'Artist': val.split('.wav')[0], 'Sample_Rate': sr, 'Wav': [i for i in y]}, ignore_index=True)
        
df.to_csv('test_wavs.csv')



In [8]:
#try to read back the wavs from the csv
df1 = pd.read_csv("test_wavs.csv")
df2 = pd.read_csv("../test_set_cleaned.csv")
merged_df = pd.merge(df1, df2, on='Artist', how='outer')
merged_df.to_csv('final_test_set.csv', index=False)
